## Import APAC Data  

In [146]:
import csv
import pandas as pd  
import matplotlib.pyplot as mpl
import xlrd

In [15]:
# load the APAC file 
AMB_FILE = '/Users/katharinechen/Desktop/AMB/AMB_ALL.tsv'
DTYPES = {'year': int,
          'quarter': object, 
          'patid': object, 
          'gender': object, 
          'agegrp': object, 
          'race': float,
          'ethnicity': float, 
          'language': object, 
          'metro': int, 
          'paid': float, 
          'payer': object, 
          'pos': object, 
          'urban': int, 
          'dx1': object, 
          'px1': object, 
          'hcg': object}
df = pd.read_csv(AMB_FILE, delimiter='\t', dtype=DTYPES)

In [17]:
df.head(5)

# year = calendar year 
# patid = encrypted unique patient id
# paycat = payer category 
# pos = place of service code 
# dx1 = principal diagnosis 
# proccode = CPT or HCPCS procedure code 
# hcg = HCG code 

,year,quarter,patid,gender,agegrp,race,ethnicity,language,metro,paid,patpaid,payer,pos,urban,dx1,px1,hcg
0,2014,Q1,YYPLRA3T0NSADX171R0K71EMJ,F,25-29,9.0,3.0,NaN,1,55.01,0.00,OTH,11,1,V7231,NaN,P40d
1,2014,Q1,K2EB2ADRWAQJADJD3UHFCSBYZ,F,20-24,9.0,3.0,NaN,1,11.98,2.99,OTH,11,1,2689,NaN,P63a
2,2014,Q1,K97GGLXYFAD46TYCL7NXP5G3,F,40-44,9.0,3.0,NaN,1,0.00,0.00,OTH,11,0,5990,NaN,P37a
3,2014,Q1,BCZYGNI7V7PX7W0U12J4TZDB,M,60-64,1.0,3.0,NaN,1,0.00,0.00,OTH,11,1,7242,NaN,P53
4,2014,Q1,K2EB2ADRWAQJADJD3UHFCSBYZ,F,20-24,9.0,3.0,NaN,1,33.55,8.39,OTH,11,1,2689,NaN,P63a


In [19]:
# ICD-9 codes (prinicipal diagnosis)
uniqueDXValue = df['dx1'].unique()
len(uniqueDXValue)

12805

In [21]:
# HCG code 
len(df['hcg'].unique())

105

## Importing ICD9 to Modules Mapping

In [25]:
# turn excel table into a pandas DataFrame
ICD9_MAPPING = '/Users/katharinechen/Desktop/icd9-mapping.xlsx'
ICD9_data = pd.read_excel(ICD9_MAPPING)

In [26]:
ICD9_data.head(5)

,ICD9,Module
0,460,CCA
1,460,CCA
2,462,CCA
3,462,CCA
4,463,CCA


In [27]:
len(ICD9_data['ICD9'].unique())

418

In [28]:
len(ICD9_data['Module'].unique())

44

In [30]:
ICD9_data['Module'].unique()

array([u'CCA', u'Rash', u'Mouth sores', u'Yeast infection',
       u'Nail problems', u'Eye pain, eye bump, or irritation', u'Ear pain',
       u'Cough, cold, or allergy (Pediatric)', u'Flu',
       u'Bladder infection', u'Diaper rash', u'Acne', u'OTHER',
       u'Burn/sunburn', u'STI exposure', u'Depression', u'Headlice',
       u'Insect bite', u'Smoking cessation', u'Hemorrhoids',
       u'Stomach pain', u'Kidney stones', u'Hair loss', u'Back pain',
       u'Headache', u'Joint pain', u'Vertigo', u'Constipation',
       u'Sleep disorders', u'Urinary incontinence', u'Contraceptives',
       u'Sexual health', u'Cholesterol management', u'Gout',
       u'Bipolar disorder', u'Depression management',
       u'Carpal tunnel syndrome', u'Blood pressure management', u'IBS',
       u'Diabetes management', u'Anxiety', u'Asthma management',
       u'Heartburn', u'Perinatal care'], dtype=object)

In [31]:
# Number of ICD9 per SmartExam Modules 
ICD9_data['Module'].value_counts()

Rash                                   111
Joint pain                              42
Burn/sunburn                            38
CCA                                     36
OTHER                                   24
Ear pain                                22
Headache                                19
Eye pain, eye bump, or irritation       13
Contraceptives                          10
Depression                              10
Insect bite                              9
Back pain                                9
Nail problems                            7
Urinary incontinence                     7
STI exposure                             7
Bladder infection                        7
Depression management                    4
Asthma management                        4
Stomach pain                             4
Yeast infection                          4
Hair loss                                4
Diabetes management                      4
Cholesterol management                   4
Headlice   

## Limited Dataset with only HCG codes "P32c" and "P33" 
P32c and P33 represents Primary Care and Urgent Care location respectively 

In [32]:
onlyHCGCondition = df['hcg'].isin(['P32c', 'P33']) 
HCG_APAC = df[onlyHCGCondition]
HCG_APAC.head(5)

,year,quarter,patid,gender,agegrp,race,ethnicity,language,metro,paid,patpaid,payer,pos,urban,dx1,px1,hcg
10,2014,Q1,PXI5514V62FPEBRV8J2AADZX,M,25-29,9.0,3.0,NaN,1,65.00,50.00,OTH,11,1,6084,*NULL*,P32c
13,2014,Q1,YB2K07ED71EIYBIBFSBYU5792RL,M,30-34,9.0,3.0,NaN,0,106.70,0.00,OTH,11,0,07999,*NULL*,P32c
16,2014,Q1,JKK6V681RIXLPMFAD7NXP5G3,F,45-49,9.0,3.0,NaN,0,0.00,138.46,OTH,11,0,24200,NaN,P32c
18,2014,Q1,MFTXR3W7XHMCH2V93TDW92RL,F,35-39,9.0,3.0,NaN,1,117.60,30.00,OTH,11,1,4660,NaN,P32c
22,2014,Q1,5NKV477WNS7PTZTX5SYMEHIQ,F,40-44,9.0,3.0,NaN,1,112.14,25.00,OTH,11,1,38870,NaN,P32c


In [33]:
# number of "Primary Care" visits  
df['hcg'].value_counts()['P32c']

7315944

In [34]:
# number of "Urgent Care" visits 
df['hcg'].value_counts()['P33']

487583

In [35]:
# Confirm that the new HCG_APAC is correct 
df['hcg'].value_counts()['P32c'] == HCG_APAC['hcg'].value_counts()['P32c'] 
df['hcg'].value_counts()['P33'] == HCG_APAC['hcg'].value_counts()['P33']

True

## Merged HCG_APAC

In [152]:
# Create a new dataframe from HCG_APAC with the only ICD9 codes that we care about  
condition = HCG_APAC['dx1'].isin([str(i) for i in ICD9_data['ICD9'].unique()])
ICD9_HCG = HCG_APAC[condition]
ICD9_HCG.head().reset_index()

,index,year,quarter,patid,gender,agegrp,race,ethnicity,language,metro,paid,patpaid,payer,pos,urban,dx1,px1,hcg
0,18,2014,Q1,MFTXR3W7XHMCH2V93TDW92RL,F,35-39,9.0,3.0,NaN,1,117.60,30.0,OTH,11,1,4660,NaN,P32c
1,22,2014,Q1,5NKV477WNS7PTZTX5SYMEHIQ,F,40-44,9.0,3.0,NaN,1,112.14,25.0,OTH,11,1,38870,NaN,P32c
2,23,2014,Q1,BCSEN77PSBR38JTZW84369WY,M,65-69,9.0,3.0,NaN,1,162.44,20.0,OTH,11,1,7804,NaN,P32c
3,50,2014,Q1,KKPNBZS7G7063607U3TDW92RL,M,45-49,9.0,3.0,NaN,1,210.63,25.0,OTH,11,0,7881,NaN,P32c
4,75,2014,Q1,N093RDKQZD482LRDPXSC7NU69WY,F,65-69,9.0,3.0,NaN,1,57.62,0.0,ADV,20,0,4739,*NULL*,P33


In [153]:
MERGED_TABLE = pd.merge(ICD9_HCG, ICD9_data, left_on=['dx1'], right_on=['ICD9'])
MERGED_TABLE.head(5)

,year,quarter,patid,gender,agegrp,race,ethnicity,language,metro,paid,patpaid,payer,pos,urban,dx1,px1,hcg,ICD9,Module
0,2014,Q1,ME8R7V7UV0847UX3T4KEBLBRW9N,F,70-74,9.0,3.0,NaN,0,23.29,0.0,ADV,11,0,V5861,*NULL*,P32c,V5861,OTHER
1,2014,Q1,5G5NG0922FLEPM2JADZYSYADZX,M,70-74,9.0,3.0,NaN,1,9.60,NaN,ADV,11,1,V5861,NaN,P32c,V5861,OTHER
2,2014,Q1,MCZHQ4ZYEHIDEQIYADZYSYADZX,M,80-84,9.0,3.0,NaN,1,9.60,NaN,ADV,11,1,V5861,NaN,P32c,V5861,OTHER
3,2014,Q1,M6GXZWCSBRW9QRKU5M712JKEHIQ,M,70-74,9.0,3.0,NaN,1,10.20,NaN,ADV,11,1,V5861,NaN,P32c,V5861,OTHER
4,2014,Q1,707UK0M1FIHYEIFPTZDPGH4K8U,F,75-79,9.0,3.0,NaN,1,10.20,NaN,ADV,11,1,V5861,NaN,P32c,V5861,OTHER


##  Data table with "gender" as columns and "module name" as rows and the cells are the total of the count of rows
 

In [164]:
gender = MERGED_TABLE[['Module', 'dx1', 'gender']]
gender.head(5)

,Module,dx1,gender
0,OTHER,V5861,F
1,OTHER,V5861,M
2,OTHER,V5861,M
3,OTHER,V5861,M
4,OTHER,V5861,F


In [165]:
gender.groupby(['gender', 'Module']).aggregate('count')

dx1
gender Module               
F      Contraceptives  19222
       Depression        683
       OTHER           21416
       Perinatal care  15023
       STI exposure      480
       Sexual health    3315
M      Contraceptives    298
       Depression        358
       OTHER           22269
       Perinatal care      2
       STI exposure      806
       Sexual health    2827

##  A data table with "age-group" as columns and "module name" as rows and the cells are the total of the count of rows


In [167]:
age = MERGED_TABLE[['Module', 'dx1', 'agegrp']]
age.head(5)

,Module,dx1,agegrp
0,OTHER,V5861,70-74
1,OTHER,V5861,70-74
2,OTHER,V5861,80-84
3,OTHER,V5861,70-74
4,OTHER,V5861,75-79


In [169]:
age.groupby(['Module', 'agegrp']).aggregate('count')

dx1
Module         agegrp      
Contraceptives 0-4        1
               10-14    545
               15-19   6108
               20-24   5058
               25-29   3386
               30-34   1987
               35-39   1238
               40-44    715
               45-49    356
               5-9        1
               50-54    113
               55-59      3
               60-64      5
               75-79      4
Depression     10-14     29
               15-19    116
               20-24     84
               25-29     86
               30-34    100
               35-39     87
               40-44     73
               45-49    107
               5-9        2
               50-54    109
               55-59     91
               60-64     78
               65-69     29
               70-74     23
               75-79      9
               80-84     12
...                     ...
STI exposure   10-14     33
               15-19     88
               20-24    220
               25-29    285
               30-34    194
               35-39    145
               40-44    108
               45-49     64
               5-9        5
               50-54     69
               55-59     31
               60-64     16
               65-69     15
               70-74      9
               75-79      4
Sexual health  10-14     57
               15-19    931
               20-24   1370
               25-29   1201
               30-34    923
               35-39    594
               40-44    413
               45-49    265
               50-54    167
               55-59    121
               60-64     44
               65-69     33
               70-74     21
               75-79      1
               80-84      1

[90 rows x 1 columns]

## Data table with "season" as columns and "module name" as rows and the cells are the total of the count of rows (Alvaro added a Q1, Q2, Q3 and Q4 label which can be used to approximate season)

In [174]:
season = MERGED_TABLE[['Module', 'dx1', 'quarter']]
season.head(5)

,Module,dx1,quarter
0,OTHER,V5861,Q1
1,OTHER,V5861,Q1
2,OTHER,V5861,Q1
3,OTHER,V5861,Q1
4,OTHER,V5861,Q1


In [175]:
season.groupby(['quarter', 'Module']).aggregate('count')

dx1
quarter Module               
Q1      Contraceptives   4264
        Depression         80
        OTHER           10753
        Perinatal care   4042
        STI exposure      267
        Sexual health    1381
Q2      Contraceptives   5120
        Depression         99
        OTHER           10902
        Perinatal care   4042
        STI exposure      347
        Sexual health    1553
Q3      Contraceptives   5038
        Depression        353
        OTHER           10889
        Perinatal care   3607
        STI exposure      350
        Sexual health    1610
Q4      Contraceptives   5098
        Depression        509
        OTHER           11141
        Perinatal care   3334
        STI exposure      322
        Sexual health    1598